In [1]:
# BASELINE
# ورود ماژول های مورد نیاز

import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import linalg
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
import time

begin_time=time.time()

# Main_Ratings مجموعه داده های امتیاز

path1="D:/PHD/رساله و نرم افزار/نرم افزار/Datasets/movielens/ml-latest-small/ml-latest-small/ratings.csv"
main_ratings = pd.read_csv(path1, delimiter=',')
main_ratings = main_ratings.drop('timestamp',axis=1)# حذف ستون زمان 
main_ratings.drop_duplicates(inplace=True)# حذف سطرهای تکراری
#display('main_ratings',main_ratings)


# Movies ورود و پردازش دیتاست فیلم ها 
path2 ='D:/PHD/رساله و نرم افزار/نرم افزار/Datasets/movielens/ml-latest-small/ml-latest-small/movies.csv'
movies = pd.read_csv(path2, delimiter=',') 
movies= movies.join(movies.genres.str.get_dummies("|")) #جدا کردن ژانرها  قرار دادن یک برای ژانرهای موجود
#display('movies:',movies)

movies_new=movies.drop(['genres','title'], axis=1)# حذف ستون های بدون ژانر و عنوان 
movies_new=movies_new.drop(movies_new.columns[1],axis=1) # حذف ستون بدون ژانر
movies_new.drop_duplicates(inplace=True)
movies_new=movies_new.set_index('movieId')
#display('movies_new:',movies_new)


# Ratings_df دیتافریم امتیازات
ratings_df=main_ratings.pivot(index='userId', columns='movieId', values='rating')# مارتریس امتیازات
ratings_df=pd.DataFrame(ratings_df)
ratings_df=ratings_df.dropna(axis='columns', how='all')# حذف کاربر بدون امتیاز
ratings_df=ratings_df.dropna(axis='rows', how='all') # حذف فیلم بدون امتیاز
display('ratings_df',ratings_df)

# ---------------حذف سطرهایی که در امتیازدهی شرکت نداشته اند
for k in movies_new.index:
    if k not in  ratings_df.columns:
        movies_new=movies_new.drop(k)
        #print(k)
display('movies_new:', movies_new)



# movies_no_ther حذف فیلم های با تعداد امتیاز کمتر از 
movies_no_ther=10
movies_count=ratings_df.count()
movies_count=movies_count[movies_count>movies_no_ther]
display('movies_count',movies_count)

# users_no_ther حذف کاربر های با تعداد امتیاز کمتر از 
users_no_ther=20
users_count=ratings_df.count(axis=1)
users_count=users_count[users_count>users_no_ther]
display('users_count',users_count)


# ماتریس کابر- فیلم پس از حذف فیلم ها و کاربران
ratings_df_new=pd.DataFrame(data=ratings_df,index=users_count.index ,columns=movies_count.index)
display('ratings_df_new',ratings_df_new)
#ratings_df_new=ratings_df_new.drop(53)
#display('ratings_df_new',ratings_df_new)

# Ratings_new دیتافریم امتیازات پس از حذف فیلم ها و کاربران با امتیاز کم
ratings_new=pd.DataFrame()

for ind in main_ratings.index:
    if (main_ratings.loc[ind].userId in ratings_df_new.index) and (main_ratings.loc[ind].movieId in ratings_df_new.columns):
        ratings_new=ratings_new.append(main_ratings.loc[ind])
display('ratings_new',ratings_new)


# train-test
train, test = train_test_split(ratings_new, test_size=0.2, random_state=100)
display('train',train,'test',test)


# train_df تبدیل داده های آموزش به دیتافریم
train_df=train.pivot(index='userId', columns='movieId', values='rating')
train_df=pd.DataFrame(train_df, index=ratings_df_new.index, columns=ratings_df_new.columns)
#display('train_df',train_df)

# test_df
#test_matrix=test.pivot(index='userId', columns='movieId', values='rating')
#test_df=pd.DataFrame(test_matrix)# , index=data_df.index, columns=data_df.columns)
#display('test_df',test_df)

#  محاسبه پیش بینی پایه Baseline

M=train_df.copy()
global_mean = M[M>0].mean().mean()
M=M.fillna(global_mean)
M_minus_mean = M[M>0]-global_mean
display('M_minus_mean',M_minus_mean)
user_bias = M_minus_mean.T.mean()
display('user_bias',user_bias)
item_bias = M_minus_mean.apply(lambda r: r - user_bias).mean()
display('item_bias',item_bias)
display('global_mean',global_mean)
display('user_bias',user_bias)
display('item_bias',item_bias)
display('M',M)
M_new = pd.DataFrame(0, index=train_df.index, columns=train_df.columns)
M_new=M_new+global_mean

M_new=M_new.apply(lambda r: r+user_bias)
M_new=(M_new.T.apply(lambda r: r+item_bias)).T
#display('M_new',M_new)# ماتریس پیش بینی با مقادیر پایه

M_1=M.fillna(M_new).round(2)


test_hat=pd.Series()
for ind in test.index:
    test_hat.loc[ind]=M_new[test.loc[ind]['movieId']][test.loc[ind]['userId'] ]

test['rating_hat']=test_hat.round(2)
display('test',test)
    
rmse_error=mean_squared_error(test['rating'], test['rating_hat'],squared=False)
mae_error=mean_absolute_error(test['rating'], test['rating_hat'])


print('Baseline Result:','mae_error=',round(mae_error,5),'rmse_error=',round(rmse_error,5))

end_time=time.time()
print('time (s)=', round(end_time-begin_time,1))


'ratings_df'

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


'movies_new:'

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193583,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193585,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


'movies_count'

...

movieId,1,2,3,5,6,7,9,10,11,12,...,159093,164179,166528,168250,168252,174055,176371,177765,179819,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,3.442684,4.000000,3.442684,4.000000,3.442684,3.442684,3.442684,3.442684,3.442684,...,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684
2,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,...,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684
3,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,...,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684
4,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,...,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684
5,4.000000,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,...,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.500000,3.442684,3.442684,3.442684,3.442684,2.500000,3.442684,3.442684,2.500000,3.442684,...,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684
607,4.000000,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.000000,3.442684,...,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684
608,2.500000,2.000000,2.000000,3.442684,3.442684,3.442684,3.442684,4.000000,3.442684,3.442684,...,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684,3.442684


C:\Users\nozar\anaconda31\lib\site-packages\ipykernel_launcher.py:123: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\nozar\anaconda31\lib\site-packages\ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'test'

,userId,movieId,rating,rating_hat
8051,57.0,110.0,4.0,3.61
9281,63.0,3949.0,5.0,3.50
80063,502.0,3147.0,5.0,3.53
21428,140.0,3998.0,4.0,3.45
95280,600.0,1197.0,5.0,3.47
...,...,...,...,...
100532,610.0,90746.0,3.5,3.54
17117,109.0,231.0,3.0,3.37
99780,610.0,3949.0,3.5,3.59
31276,217.0,2245.0,2.0,3.34


Baseline Result: mae_error= 0.78973 rmse_error= 0.98188
time (s)= 414.6
